In [7]:
import feedparser
import pandas as pd
import boto3
import json
from botocore.exceptions import ClientError
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from datetime import datetime
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
from boto3.dynamodb.conditions import Key, Attr

In [ ]:
RSS_list = [
    # --- グローバル・メガテック公式（最先端技術・基盤モデルの動向） ---
    "https://openai.com/news/rss.xml",           # OpenAI (業界のゲームチェンジャー、最新モデル発表)
    "https://cloudblog.withgoogle.com/rss/",     # Google Cloud Blog (Gemini/Vertex AI関連)
    "https://aws.amazon.com/jp/blogs/aws/feed/", # AWS News Blog (Bedrock/SageMaker関連)
    "https://developers-jp.googleblog.com/atom.xml", # Google Developers Japan (国内向け開発情報)

    # --- 国内メガベンチャー・テックブログ（実践的な実装・導入事例） ---
    "https://engineering.mercari.com/blog/feed.xml", # Mercari (大規模プロダクトへのAI実装)
    "https://developers.cyberagent.co.jp/blog/feed/", # CyberAgent (広告AI、LLM独自開発)
    "https://www.ai-shift.co.jp/techblog/feed",  # AI Shift (カスタマーサポートAI、RAG実践)
    "https://recruit-tech.co.jp/blog/feed/",     # Recruit (大規模データ活用・マッチングAI)
    "https://blog.brainpad.co.jp/rss",           # ブレインパッド (DXコンサル・データサイエンス)

    # --- 開発コミュニティ・技術メディア（現場のナレッジ・最新トレンド） ---
    "https://zenn.dev/topics/llm/feed",          # Zenn LLM (国内エンジニアの最速Tips・試行錯誤)
    "https://rss.itmedia.co.jp/rss/2.0/aiplus.xml", # ITmedia AI+ (最新ニュース・国内動向)
    "https://atmarkit.itmedia.co.jp/rss/rss.xml", # ＠IT (深掘り技術解説・マネジメント向け)
#    "https://xtech.nikkei.com/rss/xtech-it.rdf", # 日経クロステック (ビジネス視点のIT・AIニュース) #published_parsedがないかも
    "https://medium.com/feed/@kyakuno",          # Medium @kyakuno (技術選定・アーキテクチャ考察)

    # --- 論文・リサーチ（SOTA・学術的ブレイクスルー） ---
    "https://rsshub.app/huggingface/daily-papers", # Hugging Face Daily Papers (最重要論文)
    "https://research.preferred.jp/feed/",       # PFN Research (日本最高峰のAI研究)
    "https://feeds.feedburner.com/blogspot/gJZg", # Google Research Blog (グローバルな研究成果)
    "https://pytorch.org/feed.xml",              # PyTorch Official (深層学習フレームワークの進化)
#    "https://huggingface.co/blog/feed.xml",      # Hugging Face Blog (オープンソースモデルのハブ) #summaryがないかも

    # --- 海外ニュースレター・分析（戦略・マクロ・倫理） ---
    "https://www.deeplearning.ai/the-batch/rss", # The Batch (Andrew Ng氏監修の質の高い週刊分析)
    "https://www.technologyreview.jp/feed/",      # MIT Technology Review (技術の社会・倫理的インパクト)
]

#with open('RSS.json', 'w') as f:
#    json.dump(RSS_list, f, indent=2)

In [ ]:
with open('RSS.json') as f:
    RSS_list= json.load(f)

In [106]:
def parse_text(entry,today,id_1,id_2,id_3,category,news_df):
    # published_parsedをintオブジェクトに変換
    struct_time = entry.published_parsed
    dt = datetime(*struct_time[:6])
    published_datetime = dt.timestamp()
    published_datetime = int(published_datetime)

    # TTLの設定
    ttl = 14*24*60*60  # 14日間のTTLを秒単位で計算
    ttl = published_datetime + ttl

    print(f"ID: {today}{id_1}{id_2}{id_3}")
    print(f"カテゴリー: {category}")
    print(f"タイトル: {entry.title}")
    print(f"リンク: {entry.link}")
    print(f"公開日時 (datetime): {dt}")
    print(f"概要: {entry.summary}")
    print(f"TTL時間: {ttl}")

    addRow = [f"{today}{id_1}{id_2}{id_3}", category,  entry.title, entry.link, published_datetime, entry.summary, ttl]
    addRow = pd.DataFrame([addRow], columns=["id", "category", "title", "link", "published_datetime", "summary", "ttl"])
    news_df = pd.concat([news_df, addRow], ignore_index=True)
    print("-" * 30)

    if id_3 == 9:
        id_2 += 1

    if id_3 == 9:
        id_3 = 0
    else:
        id_3 += 1

    return news_df, id_2, id_3

In [108]:
def get_news_data(RSS_list):
    id_1 = 1
    today = datetime.now().strftime("%Y%m%d")
    category = "AI_news"
    news_df = pd.DataFrame()

    for url in RSS_list:
        feed = feedparser.parse(url)
        id_2 = 0
        id_3 = 1
        for entry in feed.entries[0:1]:
            parse_text(entry,today,id_1,id_2,id_3,category,news_df)
        id_1 += 1

    return news_df

In [ ]:
news_df = get_news_data(RSS_list)
# HTMLタグを除去する関数
def clean_html(html, strip=False):
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text(strip=strip)
    return text
news_df["summary"] = news_df["summary"].apply(lambda x: clean_html(x,strip=True))

In [ ]:
table_name = 'test-project'
region_name = 'ap-northeast-1'

# 一括書き込み。batch_writerを使用すると、DynamoDBの制限に基づいて自動的にバッチを分割してくれる。
def dynamo_batch_write(news_df, table_name, region_name='ap-northeast-1'):
    news_data = news_df.to_dict(orient='records')
    #JSONファイルの読み込み
    # dynamodbの設定
    dynamodb = boto3.resource('dynamodb', region_name=region_name)
    table = boto3.resource('dynamodb').Table(table_name)

    with table.batch_writer() as batch:
        for item in news_data:
            item_not_has_nan = {key: item[key] for key in item if item[key] is not np.nan}
            batch.put_item(Item=item_not_has_nan)
    return "Successfully wrote to DynamoDB"

dynamo_batch_write(news_df, table_name, region_name)

In [ ]:
def get_dynamo_data(days=7, table_name='test-project'):
    ut = time.time()
    START_TIME = int(ut - days*24*60*60) # 7日間の範囲指定
    END_TIME = int(ut) #現在の時間

    client = boto3.client('dynamodb')
    dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-1')
    table = boto3.resource('dynamodb').Table(table_name)

    response = table.query(
        IndexName = "published_datetime",
        KeyConditionExpression=(
            Key("category").eq("AI_news") &
            Key("published_datetime").between(START_TIME, END_TIME)
        )
    )

    items = response.get('Items', [])
    return items

1769862495
1770467295


[{'summary': 'For any organization that has invested in an internal developer platform, a question inevitably arises: Is it actually working?Simply tracking adoption rates won\'t tell you if your platform is truly delivering value to your developers. This was the challenge faced by John Lewis, a major UK retailer. In our previous articles (parts1and2) we introduced the John Lewis Digital Platform (JLDP) and how it enabled dozens of product teams to build high-quality software rapidly to power www.johnlewis.com and other critical applications. But how did they know that the platform was actually successful? Traditional product metrics like revenue and sales don’t translate easily to this world. When you focus only on whether your tenants use the platform, you don’t understand whether it’s bringing them value.In this article, Alex Moss from the John Lewis platform team discusses how they moved beyond simple usage metrics to develop a sophisticated, multi-stage approach to measuring the r

In [ ]:
def load_api_key():
    load_dotenv(dotenv_path="../.env", override=True)
    os.environ["AWS_BEARER_TOKEN_BEDROCK"] = os.getenv("Bedrock_API_Key")

def create_response(prompt_text,news_data):
    client = boto3.client("bedrock-runtime")

    prompt = PromptTemplate(
        input_variables=["news_data"],
        template = prompt_text)
    prompt = prompt.format(news_data=news_data)

    body = json.dumps({
        "messages":[
            {"role":"user","content":prompt}
        ],
        "max_tokens":4096,
        "temperature":0.5,
        "anthropic_version":"bedrock-2023-05-31"
    })

    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

    response = client.invoke_model(
        modelId=model_id,
        body=body
    )

    return response

load_api_key()

f = open("prompt.txt","r",encoding="utf-8")
prompt_text = f.read()
news_data = get_dynamo_data(days=7, table_name='test-project')
response = create_response(prompt_text,news_data)

response_body = json.loads(response.get("body").read())
answer = response_body["content"][0]["text"]
print(answer)

[
  {
    "link": "https://openai.com/index/introducing-openai-frontier",
    "category": "Tech/Library",
    "priority": "High",
    "reason": "OpenAIが企業向けのAIエージェント開発・管理プラットフォーム「OpenAI Frontier」を発表しました。これは共有コンテキスト、オンボーディング、権限管理、ガバナンスなどの機能を備えており、企業のAI導入を大きく促進する可能性があります。AIの実用化に直結する重要な発表であり、開発者やプロジェクトマネージャーにとって必読の情報です。"
  },
  {
    "link": "https://cloud.google.com/blog/topics/developers-practitioners/getting-started-with-gemini-3-unlocking-the-cloud-with-the-free-trial/",
    "category": "Tech/Library",
    "priority": "Medium",
    "reason": "Google CloudのGemini 3の無料トライアルについての解説記事です。AI開発者にとって有用な情報が含まれていますが、特に革新的な内容ではないため、優先度はMediumとしました。Google CloudでのAIアプリケーション開発に興味のある開発者にとっては参考になる情報です。"
  },
  {
    "link": "https://aws.amazon.com/blogs/aws/amazon-ec2-c8id-m8id-and-r8id-instances-with-up-to-22-8-tb-local-nvme-storage-are-generally-available/",
    "category": "Tech/Library",
    "priority": "Medium",
    "reason": "AWSが新しいEC2インスタンスタイプを発表しました。最大22.8TBのNVMeストレージを備え、高性能なコンピューティングリソースを提供

### 今はいらないけど本文スクレイピングのコード

In [ ]:
def fetch_article_text(url):
    try:
        # 1. サーバーに負荷をかけすぎないよう配慮しつつ取得
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
        response = requests.get(url, headers=headers, timeout=10)
        response.encoding = response.apparent_encoding # 文字化け防止
        
        soup = BeautifulSoup(response.text, "html.parser")

        # 2. 不要な要素を事前に削除（ノイズ除去率 約40%向上）
        for tag in soup(["script", "style", "nav", "header", "footer", "aside", "form"]):
            tag.decompose()

        # 3. 汎用的な本文抽出ロジック
        # 多くのテックブログは <article> または <main> タグ内に本文がある
        content_area = soup.find('article') or soup.find('main')
        
        if not content_area:
            # article/mainがない場合、最もテキスト密度が高いdivを探す（予備ロジック）
            # ここではシンプルに body 全体から取得
            content_area = soup.find('body')

        if content_area:
            # テキストを抽出し、余計な空白を削除
            lines = (line.strip() for line in content_area.get_text(separator="\n").splitlines())
            text = "\n".join(chunk for chunk in lines if chunk)
            return text
        
        return "本文が取得できませんでした。"

    except Exception as e:
        return f"エラーが発生しました: {e}"

# --- 実行部分（汎用ループ） ---
# news_json[n] に対して実行
for n in range(len(news_data)):
    target_url = news_data[n]["link"]
    print(f"解析中: {target_url}")

    # 本文を抽出して格納
    news_data[n]["text"] = fetch_article_text(target_url)

    # 定量的確認：抽出した文字数
    print(f"抽出完了：{len(news_data[n]['text'])} 文字")
    print("-" * 30)